# Neural Network Model 

In [1]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, Input
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import timedelta
import glob

In [3]:
# Φόρτωσε όλα τα αρχεία CSV στον κατάλογο σε μια λίστα με DataFrames
csv_files = glob.glob('harth2/*.csv')

# Κατασκευή του μοντέλου μας 
model = Sequential()
model.add(Dense(12 * 8, input_shape=(306,), activation=LeakyReLU(alpha=0.1)))
model.add(Dense(12 * 4, activation=LeakyReLU(alpha=0.1)))
model.add(Dense(12 * 2, activation=LeakyReLU(alpha=0.1)))
model.add(Dense(13, activation='softmax'))

# Σύνταξη του μοντέλου μας 
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Αρχικοποίηση λιστών για την αποθήκευση των μετρικών αξιολόγησης
accuracies_NN = []
precisions_NN = []
recalls_NN = []
f1s_NN = []

for csv_file in csv_files:
    # Ανάγνωση του αρχείου CSV
    df = pd.read_csv(csv_file)

    lagged = df.copy()
    lagged_columns = []

    # Δημιουργία καθυστερημένων στηλών
    lagged_col_data = {}  # Λεξικό για αποθήκευση δεδομένων καθυστερημένων στηλών
    for i in range(1, 51):
        for col in ['back_x', 'back_y', 'back_z', 'thigh_x', 'thigh_y', 'thigh_z']:
            lagged_col_name = f"{col}_{i}"  # Δημιουργία ονόματος καθυστερημένης στήλης
            lagged_col_data[lagged_col_name] = lagged[col].shift(i)  # Δημιουργία καθυστερημένων στηλών
            lagged_columns.append(lagged_col_name)

    # Συγχώνευση δεδομένων καθυστερημένων στηλών
    lagged = pd.concat([lagged, pd.DataFrame(lagged_col_data)], axis=1)

    # Συμπλήρωση τιμών NaN με 0
    lagged.fillna(0, inplace=True)

    # Αφαίρεση της στήλης 'label' από το DataFrame
    label_column = lagged.pop('label')

    # Εισαγωγή της στήλης 'label' στο τέλος του νέου Data Frame 
    lagged['label'] = label_column

    # Aντιστοίχιση τιμων Label 
    mapping = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 13: 9, 14: 10, 130: 11, 140: 12}
    # Αντικατάσταση τιμών
    lagged['label'] = lagged['label'].replace(mapping)

    # Εμφάνιση του πρώτου κομματιού του καθυστερημένου DataFrame για έλεγχο των αλλαγών
    lagged.head()

    # Αφαίρεση της στήλης 'time_only' από το νέο DataFrame
    lagged = lagged.drop(['time_only'], axis=1)

    # Διαχωρισμός χαρακτηριστικών (X) και ετικετών (y)
    X = lagged.iloc[:, :-1].values  # Εξαίρεση της τελευταίας στήλης, που είναι η ετικέτα
    y = lagged.iloc[:, -1].values  # Λήψη μόνο της τελευταίας στήλης ως ετικέτες

    # Κανονικοποίηση των χαρακτηριστικών
    sc = StandardScaler()
    X = sc.fit_transform(X)

    # Διαχωρισμός του συνόλου δεδομένων σε εκπαιδευτικό και δοκιμαστικό σύνολο
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Εκπαίδευση του μοντέλου
    history = model.fit(X_train, y_train, epochs=50, batch_size=30)

    # Υπολογισμός ακρίβειας του μοντέλου 
    score = model.evaluate(X_test, y_test, verbose=0)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    
    # Προσθήκη της ακρίβειας στη λίστα των ακριβειών
    accuracy = score[1]
    accuracies_NN.append(accuracy)

    # Πρόβλεψη πιθανοτήτων για το test set
    yhat_probs = model.predict(X_test, verbose=0)

    # Πρόβλεψη κλάσεων για το test set
    classes_x = np.argmax(yhat_probs, axis=1)

    # Ακρίβεια πρόβλεψης: tp / (tp + fp)
    precision = precision_score(y_test, classes_x, average='weighted')
    print("Precision: %f" % precision)
    precisions_NN.append(precision)

    # Ανάκληση: tp / (tp + fn)
    recall = recall_score(y_test, classes_x, average='weighted')
    print("Recall: %f" % recall)
    recalls_NN.append(recall)

    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, classes_x, average='weighted')
    print("F1 score: %f" % f1)
    f1s_NN.append(f1)

# Υπολογισμός του μέσου όρου της ακρίβειας, ακρίβειας πρόβλεψης, ανάκλησης και f1 σε όλα τα αρχεία
overall_accuracy = sum(accuracies_NN) / len(accuracies_NN)
print(f'Overall Accuracy: {overall_accuracy}')
overall_precision = sum(precisions_NN) / len(precisions_NN)
print(f'Overall Precision: {overall_precision}')
overall_recall = sum(recalls_NN) / len(recalls_NN)
print(f'Overall Recall: {overall_recall}')
overall_f1 = sum(f1s_NN) / len(f1s_NN)
print(f'Overall F1: {overall_f1}')

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step - accuracy: 0.9063 - loss: 0.2729
Epoch 2/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9392 - loss: 0.1678
Epoch 3/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 22s 1ms/step - accuracy: 0.9467 - loss: 0.1444
Epoch 4/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9503 - loss: 0.1338
Epoch 5/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.9535 - loss: 0.1257
Epoch 6/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9556 - loss: 0.1180
Epoch 7/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9568 - loss: 0.1137
Epoch 8/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.9593 - loss: 0.1077
Epoch 9/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9599 - loss: 0.1051
Epoch 10/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9615 - loss: 0.1020
Epoch 11/50
10899/10899 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.9629 - loss: 0.09

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8849 - loss: 7.2996
Epoch 2/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9426 - loss: 0.2645
Epoch 3/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9556 - loss: 0.1517
Epoch 4/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9614 - loss: 0.1142
Epoch 5/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9657 - loss: 0.0988
Epoch 6/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9717 - loss: 0.0803
Epoch 7/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9744 - loss: 0.0719
Epoch 8/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9774 - loss: 0.0621
Epoch 9/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9789 - loss: 0.0562
Epoch 10/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9819 - loss: 0.0464
Epoch 11/50
4120/4120 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9822 - loss: 0.0500
Epoch 12/50
4120/4120 ━━━━━━━━

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Epoch 1/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.9487 - loss: 0.6173
Epoch 2/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9603 - loss: 0.1344
Epoch 3/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9622 - loss: 0.2673
Epoch 4/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.9652 - loss: 0.1100
Epoch 5/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step - accuracy: 0.9675 - loss: 0.1009
Epoch 6/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9675 - loss: 0.1162
Epoch 7/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9690 - loss: 0.1046
Epoch 8/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9705 - loss: 0.1046
Epoch 9/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9717 - loss: 0.0892
Epoch 10/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.9709 - loss: 0.0842
Epoch 11/50
9478/9478 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.9710 - loss: 0.1344
Epoch 12/50
9478/94

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.8630 - loss: 0.8194
Epoch 2/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.8865 - loss: 0.4876
Epoch 3/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.8918 - loss: 0.3496
Epoch 4/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.8941 - loss: 0.3282
Epoch 5/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.8960 - loss: 0.3100
Epoch 6/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.8996 - loss: 0.3317
Epoch 7/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.8995 - loss: 0.3525
Epoch 8/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9011 - loss: 0.2914
Epoch 9/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.9022 - loss: 0.3650
Epoch 10/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9036 - loss: 0.2738
Epoch 11/50
8594/8594 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9042 - loss: 0.2752
Epoch 12/50
8594/85

C:\Users\karag\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.8838 - loss: 3.6915
Epoch 2/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9228 - loss: 0.4372
Epoch 3/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9336 - loss: 0.3062
Epoch 4/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9385 - loss: 0.2319
Epoch 5/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9444 - loss: 0.1965
Epoch 6/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9479 - loss: 0.1659
Epoch 7/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9504 - loss: 0.1529
Epoch 8/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9540 - loss: 0.1385
Epoch 9/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9535 - loss: 0.1416
Epoch 10/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9561 - loss: 0.1300
Epoch 11/50
5205/5205 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9598 - loss: 0.1150
Epoch 12/50
5205/5205 ━━━━━━━